In [1]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from datasets import Dataset

import pandas as pd
import numpy as np

from tqdm import tqdm
import torch
import json

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Overview

In [2]:
train_df = pd.read_csv('/project/lt200304-dipmt/paweekorn/data/train_sampling.csv')

with open('/project/lt200304-dipmt/paweekorn/data/WIPO.json', 'r') as f:
    wipo_data = json.load(f)
    wipo_data = {int(k): v for k, v in wipo_data.items()}

train_df['WIPO'] = train_df['NAME'].map(wipo_data)
print(train_df.shape)
train_df.head()

(16338, 4)


,NAME,ENG,THA,WIPO
0,25,clothing for motorists and travellers,เครื่องแต่งกายสำหรับผู้ขับขี่มอเตอร์ไซค์และนัก...,"Clothing, footwear, headwear."
1,25,thermal suits,ชุดสูทที่สามารถรักษาอุณหภูมิ,"Clothing, footwear, headwear."
2,25,"water sports suits, namely wetsuits and dry su...","ชุดเล่นกีฬาทางน้ำ ได้แก่ เวทสูท, และดรายสูทสำห...","Clothing, footwear, headwear."
3,25,"boots, not for sports",รองเท้าบูทที่ไม่ใช้ใส่เล่นกีฬา,"Clothing, footwear, headwear."
4,25,fancy-dress costumes,ชุดงานแฟนซี,"Clothing, footwear, headwear."


In [3]:
model_id = "gemma3-12b-it"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = f"/project/lt200304-dipmt/paweekorn/models/base/{model_id}",
    max_seq_length = 1024,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
    device_map="auto",
)

==((====))==  Unsloth 2025.8.4: Fast Gemma3 patching. Transformers: 4.55.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.496 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


<string>:37: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [4]:
!nvidia-smi

Tue Aug 26 04:42:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  |   00000000:C1:00.0 Off |                    0 |
| N/A   36C    P0             56W /  400W |   12667MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## Data Prep

In [5]:
tokenizer = get_chat_template(tokenizer, "gemma-3")

instruction = """## Instructions:
You are an expert in the classification of goods and services under the WIPO Nice Classification system. Your task is to translate product names from English to accurate and direct Thai.

## Translation Guidelines:
- Maintain the original format of the input text.
- Use Thai legal and commercial terminology appropriate for trademarks and product classification.
- Answer in Thai language only.
- Do not include explanations, commentary, or any information beyond the translation output.

## Product Domain:
{}

## Example:
Input: material for electricity mains (wires, cables)
Output: วัสดุสำหรับส่วนควบคุมไฟฟ้าหลัก (ลวด สายเคเบิล)

## Source Text:
{}
"""

def formatting_prompt(df):
    batch = []
    for _, row in tqdm(df.iterrows()):
        src, dest = row['ENG'], row['THA']
        prompt = [
            { "role": "user", "content": instruction.format(row['WIPO'], src) }, 
            { "role": "assistant", "content": dest }
        ]
        message = tokenizer.apply_chat_template(prompt, tokenize=False)
        batch.append({'text': message})

    return Dataset.from_list(batch)

train_set = formatting_prompt(train_df)
train_set = train_set.shuffle(seed=42)
print(train_set['text'][0])

16338it [00:01, 10273.74it/s]


<bos><start_of_turn>user
## Instructions:
You are an expert in the classification of goods and services under the WIPO Nice Classification system. Your task is to translate product names from English to accurate and direct Thai.

## Translation Guidelines:
- Maintain the original format of the input text.
- Use Thai legal and commercial terminology appropriate for trademarks and product classification.
- Answer in Thai language only.
- Do not include explanations, commentary, or any information beyond the translation output.

## Product Domain:
Chemicals for use in industry, science and photography, as well as in agriculture, horticulture and forestry; unprocessed artificial resins, unprocessed plastics; fire extinguishing and fire prevention compositions; tempering and soldering preparations; substances for tanning animal skins and hides; adhesives for use in industry; putties and other paste fillers; compost, manures, fertilizers; biological preparations for use in industry and scien

## Model Training

In [6]:
model = FastLanguageModel.get_peft_model(
    model, 
    r = 8,           
    lora_alpha = 8,  
    lora_dropout = 0,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "lm_head"],
    # target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias = "none",
    random_state = 3407,
)

/home/psoratya/.conda/envs/unsloth_env/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:693: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients


In [8]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_set,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 32,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        num_train_epochs = 1,
        # max_steps = 30,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
    ),
)

train_stats = trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/16338 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 16,338 | Num Epochs = 1 | Total steps = 256
O^O/ \_/ \    Batch size per device = 32 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (32 x 2 x 1) = 64
 "-____-"     Trainable parameters = 14,238,208 of 12,201,563,248 (0.12% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,4.080200
2,4.037300
3,4.140000
4,4.049100
5,3.734100
6,3.428200
7,3.131700
8,2.921100
9,2.684200
10,2.510000


In [11]:
save_dir = "/project/lt200304-dipmt/paweekorn/models"
save_method = "merged_16bit"

# merged_model
# model.save_pretrained_merged(f"{save_dir}/fine-tuned/{model_id}", tokenizer, save_method=save_method,)

# save just lora adapter
model.save_pretrained(f"{save_dir}/adapter/{model_id}", save_method=save_method)
tokenizer.save_pretrained(f"{save_dir}/adapter/{model_id}", save_method=save_method)

['/project/lt200304-dipmt/paweekorn/models/adapter/gemma3-12b-it/processor_config.json']